<a href="https://colab.research.google.com/github/SashaNasonova/snowStudy/blob/main/gee_sentinel2_albedo_ndsi_ndwi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Snow Study Notebook

This notebook was created to generate albedo, ndsi and ndwi rasters for a time-series albedo study that aims to track albedo changes through time. The methods are based on Hammaer et al, 2023 and are as follows:

- Search Sentinel-2 Surface Reflectance archive
- Mask clouds using s2cloudless (40% probability, 0.15 NIR, 50m buffer, max distance of 2 km from cloud edge for cloud shadow search)
- Bands: blue - b2, green - b3, red - b4, NIR - b8, swir1 - b11, swir2 - b12
- Albedo =  0.356b2  + 0.130b4 + 0.373b8 + 0.085b11 + 0.072b12 - 0.0018
- NDSI = (b3 - b11) / (b3 + b11)
- NDWI = (b3 - b8)/ (b3 + b8)
- Export cloud raster (1 for cloud)
- Export truecolor/fcir 3-band images for visualization


In [1]:
#Install libraries
%pip install geemap
%pip install pycrs rasterio python-pptx cartopy requests

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 29.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 472.8/472.8 kB 20.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.8/11.8 MB 27.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 175.3/175.3 kB 5.2 MB/s eta 0:00:00
  Created wheel for pycrs: filename=PyCRS-1.0.2-py3-none-any.whl size=32686 sha256=46e3a6c3d5ec0167c49d114da96f58f667859149820839b8b37b7ca02a72b135
  Stored in directory: /root/.cache/pip/wheels/b5/4a/72/1ba05f57ddf2cc80ad21a26512097762561d646ff3ff85f729
Successfully built pycrs


In [22]:
#Import libraries
import os, shutil
import ee, geemap
import geopandas
from google.colab import files

### Define functions
Define albedo, ndsi, ndwi and cloud masking functions here.

In [3]:
## Define functions
# Helper function to get files, non-recursive
def getfiles(d,ext):
  paths = []
  for file in os.listdir(d):
      if file.endswith(ext):
          paths.append(os.path.join(d, file))
  return(paths)

# Helper function to get image acquisition date and format into ("yyyy-mm-dd")
def getDate(im):
  return(ee.Image(im).date().format("YYYY-MM-dd"))

# Helper function to get scene ids
def getSceneIds(im):
  return(ee.Image(im).get('PRODUCT_ID'))

# Functions to mosaic by image date
def mosaicByDate(indate):
  d = ee.Date(indate)
  #print(d)
  im = col.filterBounds(poly).filterDate(d, d.advance(1, "day")).mosaic()
  #print(im)
  return(im.set("system:time_start", d.millis(), "system:index", d.format("YYYY-MM-dd")))

def runDateMosaic(col_list):
  #get a list of unique dates within the list
  date_list = col_list.map(getDate).getInfo()
  udates = list(set(date_list))
  udates.sort()
  udates_ee = ee.List(udates)

  #mosaic images by unique date
  mosaic_imlist = udates_ee.map(mosaicByDate)
  return(ee.ImageCollection(mosaic_imlist))

# Multiplies Sentinel-2 imagery by 0.0001
def apply_scale_factors_s2(image):
  opticalBands = image.select('B.*').multiply(0.0001)
  return image.addBands(opticalBands, None, True)

# Calculate NDSI (Sentinel-2)
def NDSI_S2(image):
  ndsi = image.expression(
      '(Green - SWIR1) / (Green + SWIR1)', {
          'Green': image.select('B3'),
          'SWIR1': image.select('B11')}).rename('ndsi')
  return(ndsi)

# Calculate NDWI (Sentinel-2)
def NDWI_S2(image):
  ndwi = image.expression(
      '(Green - NIR) / (Green + NIR)', {
          'Green': image.select('B3'),
          'NIR': image.select('B8')}).rename('ndwi')
  return(ndwi)

# Calculate albedo (Sentinel-2), this will be 20m because of SWIR bands
def albedo_S2(image):
  #0.356b2 + 0.130b4 + 0.373b8 + 0.085b11 + 0.072b12 - 0.0018
  albedo = image.expression(
      '0.356*Blue + 0.130*Red + 0.373*NIR + 0.085*SWIR1 + 0.072SWIR2 - 0.0018', {
          'Blue': image.select('B2'),
          'Red': image.select('B4'),
          'NIR': image.select('B8'),
          'SWIR1':image.select('B11'),
          'SWIR2': image.select('B12')}).rename('albedo')
  return(albedo)

# Tiling function, uses a geometry (footprint) to split into a defined
# number or rows and columns (nx,ny)
def grid_footprint(footprint,nx,ny):
  from shapely.geometry import Polygon, LineString, MultiPolygon
  from shapely.ops import split

  #polygon = footprint
  polygon = Polygon(footprint['coordinates'][0])
  #polygon = Polygon(footprint)

  minx, miny, maxx, maxy = polygon.bounds
  dx = (maxx - minx) / nx  # width of a small part
  dy = (maxy - miny) / ny  # height of a small part

  horizontal_splitters = [LineString([(minx, miny + i*dy), (maxx, miny + i*dy)]) for i in range(ny)]
  vertical_splitters = [LineString([(minx + i*dx, miny), (minx + i*dx, maxy)]) for i in range(nx)]
  splitters = horizontal_splitters + vertical_splitters

  result = polygon
  for splitter in splitters:
      result = MultiPolygon(split(result, splitter))

  coord_list = [list(part.exterior.coords) for part in result.geoms]

  poly_list = []
  for cc in coord_list:
      p = ee.Geometry.Polygon(cc)
      poly_list.append(p)
  return(poly_list)

def aoionly(img):
  return(img.updateMask(poly_mask))

In [6]:
# Authenticate gee
ee.Authenticate()

True

In [7]:
# Initialize with google cloud project
project = 'snow-study'
ee.Initialize(project=project)

In [8]:
# Download area of interest from BCBox (make sure it's set to public)
# Open fires shapefile if exists
aoi_shp = '/content/sf_poly_sub.shp'
print(aoi_shp)

/content/sf_poly_sub.shp


In [9]:
# Visualize table and select polygon
import warnings
from google.colab import data_table
warnings.filterwarnings("ignore")
data_table.enable_dataframe_formatter()

# Visualize in table format
poly = geemap.shp_to_ee(aoi_shp)
poly_df = geopandas.read_file(aoi_shp)
poly_df_tbl = poly_df.drop(columns=['geometry'], axis=1, inplace=False)
poly_df_tbl

,Shape_Leng,Shape_Area,area_ha
0,40413.937228,9.769494e+07,9769.493692


In [10]:
# Add to map to visualize
Map = geemap.Map()
Map.addLayer(poly,{},'Area of Interest')
Map.centerObject(poly,zoom=12)
Map

Map(center=[50.171457209633836, -114.80640569322443], controls=(WidgetControl(options=['position', 'transparen…

In [12]:
# Create output folder
aoi_str = 'sf_poly_sub'

if not os.path.exists(aoi_str):
  os.mkdir(aoi_str)

# Save a copy of the polygon in a vectors folder
vector_folder = os.path.join(aoi_str,'vectors')
if not os.path.exists(vector_folder):
  os.mkdir(vector_folder)
out_shp = os.path.join(vector_folder,aoi_str+'.shp')
poly_df.to_file(out_shp,driver='ESRI Shapefile')

# Create raster mask to reduce extent of image collections
# Function aoionly in functions
poly_buf = poly.geometry().buffer(500).bounds()
poly_mask = ee.Image.constant(1).clip(poly_buf).selfMask()

In [15]:
# Select collection, Sentinel-2 Surface Reflectance. Flexible to add other data sources.
dattype = 'S2'
cld_field =  'CLOUDY_PIXEL_PERCENTAGE'
col_name = 'COPERNICUS/S2_SR_HARMONIZED'

col = ee.ImageCollection(col_name)
print(col_name,'selected')

COPERNICUS/S2_SR_HARMONIZED selected


In [17]:
# Search imagery
startdate = '2023-09-01'
enddate = '2023-10-01'
cld_thr = 70

col_filtered = col.filterDate(startdate,enddate).filterBounds(poly).filter(ee.Filter.lt(cld_field,cld_thr)).map(aoionly)
col_filtered_size = col_filtered.size().getInfo() #get size of the collection

col_filtered_list = col_filtered.toList(col_filtered_size)
col_filtered_mosaic = runDateMosaic(col_filtered_list)
col_mosaic = col_filtered_mosaic.map(apply_scale_factors_s2)

print('Found',col_filtered_size,'images')

Found 3 images


In [19]:
# Calculate indices and albedo
ndsi = col_mosaic.map(NDSI_S2)
ndwi = col_mosaic.map(NDWI_S2)
albedo = col_mosaic.map(albedo_S2)

In [24]:
# Export rasters: ndwi, ndsi, albedo, truecolor rgb, fcir rgb, cloud
crs_out = 'EPSG:3005'
outfolder = aoi_str
px_ind = 20
px_rgb = 10

folders = [
    'ndsi',
    'ndwi',
    'albedo',
    'truecolor',
    'falsecolor']

for folder_name in folders:
    folder_path = os.path.join(outfolder, folder_name)
    if os.path.exists(folder_path):
        print(f"{folder_path} exists. Deleting.")
        shutil.rmtree(folder_path)
    os.makedirs(folder_path)

## Define tiling rules
poly_area = round(poly.geometry().area(1).divide(10000).getInfo(),1)
print(poly_area)
print('Poly Area:',poly_area,'hectares')

if poly_area < 10000:
    n = 2
elif poly_area >= 10000 and poly_area < 100000:
    n = 3
elif poly_area >= 100000 and poly_area < 400000:
    n = 4
elif poly_area >= 400000 and poly_area < 1000000:
    n = 5
else:
    n = 6

print('Number of tiles: ' + str(n*n))

## Create tiles
footprint = poly.geometry().bounds().getInfo()
grids = grid_footprint(footprint,n,n)

## Export NDSI
for i in range(0,len(grids)):
  roi = grids[i]
  geemap.ee_export_image_collection(ndsi,out_dir=os.path.join(outfolder,'ndsi',str(i)), scale=px_ind, region=roi,file_per_band=True)

# Recursively get all filepaths inside ndsi subfolder and mosaic

## Export NDWI

# Recursively get all filepaths inside ndwi subfolder and mosaic

## Export truecolor 3-band image

# Recursively get all filepaths inside tc subfolder and mosaic

## Export fcir 3-band image

# Recursively get all filepaths inside fcir subfolder and mosaic

sf_poly_sub/ndsi exists. Deleting.
sf_poly_sub/ndwi exists. Deleting.
sf_poly_sub/albedo exists. Deleting.
sf_poly_sub/truecolor exists. Deleting.
sf_poly_sub/falsecolor exists. Deleting.
9736.2
Poly Area: 9736.2 hectares
Number of tiles: 4
Total number of images: 3

Exporting 1/3: sf_poly_sub/ndsi/0/2023-09-08.tif
Generating URL ...
Please wait ...
Data downloaded to /content/sf_poly_sub/ndsi/0


Exporting 2/3: sf_poly_sub/ndsi/0/2023-09-18.tif
Generating URL ...
Please wait ...
Data downloaded to /content/sf_poly_sub/ndsi/0


Exporting 3/3: sf_poly_sub/ndsi/0/2023-09-28.tif
Generating URL ...
Please wait ...
Data downloaded to /content/sf_poly_sub/ndsi/0


Total number of images: 3

Exporting 1/3: sf_poly_sub/ndsi/1/2023-09-08.tif
Generating URL ...
Please wait ...
Data downloaded to /content/sf_poly_sub/ndsi/1


Exporting 2/3: sf_poly_sub/ndsi/1/2023-09-18.tif
Generating URL ...
Please wait ...
Data downloaded to /content/sf_poly_sub/ndsi/1


Exporting 3/3: sf_poly_sub/ndsi/1/2023-0

In [ ]:
#Zip to download
zipped = outfolder + '.zip'

!zip -r {zipped} {outfolder}
files.download(zipped)